In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sn
%matplotlib inline
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = pd.read_csv("/home/growlt199/Downloads/manufacturing_project/data/flight_processed_data_wo_stops.csv",index_col=0)

In [3]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price,travel_route
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953,Delhi-Mumbai
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953,Delhi-Mumbai
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956,Delhi-Mumbai
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955,Delhi-Mumbai
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955,Delhi-Mumbai


In [4]:
#removing colums which i dont want to use in model 
df1= df.drop(['flight', 'travel_route'],axis=1)

In [5]:
# get dummy variables for multiple columns
dummy_cols = ['airline', 'departure_time','arrival_time','source_city','destination_city','stops','class']
df1_dummy = pd.get_dummies(df1[dummy_cols])

# concatenate original DataFrame with dummy variable DataFrame
df1_concat = pd.concat([df1.drop(dummy_cols, axis=1), df1_dummy], axis=1)

# display result
df1_concat.head()

,duration,days_left,price,airline_AirAsia,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,departure_time_Afternoon,...,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai,stops_one,stops_two_or_more,stops_zero,class_Business,class_Economy
0,2.17,1,5953,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
1,2.33,1,5953,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
2,2.17,1,5956,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
3,2.25,1,5955,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,1
4,2.33,1,5955,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,1


In [6]:
#create x and y where x are features for the model and y as target
x = df1_concat.drop('price',axis=1)
y = df1_concat['price']

# hyper parameter tunning

In [7]:
param_grid = {
    'n_estimators': [200, 500],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Create a random forest regression model
rf = RandomForestRegressor()

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(x, y)

# Get the best parameter combination
best_params = grid_search.best_params_


In [9]:
best_params

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200}

In [10]:
def regression(x1,y1):
    import time

    start_time = time.time()


    #use sklearn train test spilt for spilt our data in parts like training data and testing data 
    from sklearn.model_selection import train_test_split

    x_train, x_test, y_train, y_test = train_test_split(x1,y1,test_size=0.3, random_state =101, shuffle=True)

    #print shape of x train,y train, x test and y test 
    print('x_train shape:-',x_train.shape)
    print('x_test shape:-',x_test.shape)
    print('y_train shape:-',y_train.shape)
    print('y_test shape:-',y_test.shape)


    #import model from sklearn
    from sklearn.ensemble import RandomForestRegressor
    print('model name :- random forest regression')

    regr = RandomForestRegressor(max_depth=None,max_features='sqrt',min_samples_leaf=2,min_samples_split=2,n_estimators=200,random_state=123)

    #training model input as x_train and y_tarin
    regr.fit(x_train,y_train)

    print('score on the testing data :-',regr.score(x_test,y_test))
    print('score on the training data :-',regr.score(x_train,y_train))

    #finding error methods 
    from sklearn import metrics

    #model is creted show give x test data as input and retuen predictd data 
    predictions = regr.predict(x_test)

    #MAE
    print('MAE:-',metrics.mean_absolute_error(y_test,predictions))

    #MSE
    print('MSE:-',metrics.mean_squared_error(y_test,predictions))

    #RMSE
    print('RMSE:-',np.sqrt(metrics.mean_squared_error(y_test,predictions)))

    #r2
    print("r2",r2_score(y_test, predictions))

    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")
    
    return y_test,predictions

In [12]:
y_test,predictions = regression(x,y)

x_train shape:- (210100, 37)
x_test shape:- (90044, 37)
y_train shape:- (210100,)
y_test shape:- (90044,)
model name :- random forest regression
score on the testing data :- 0.9851264781557401
score on the training data :- 0.9925212989049406
MAE:- 1269.9349856460667
MSE:- 7655319.5140777845
RMSE:- 2766.8248072615265
r2 0.9851264781557401
Execution time: 37.76533222198486 seconds
